In [1]:
import pandas as pd
from sqlite3 import connect
from datetime import datetime as dt
from IPython.display import Image, display

In [2]:
conn = connect('history.db')

In [ ]:
conn.close()

In [3]:
# RMA LIST FROM MASTER PENDING
rma_list = []
q = '''
        SELECT rma FROM new_ml
    '''
try:
    rl = pd.read_sql(q,conn)
#     display(rl)
    for rma in rl['rma']:
        rma_list.append(rma)
except Exception as e:
    
    print(e)
len(rma_list)

194

In [4]:
# RMA LIST FROM TRANSFER
q = '''
        SELECT rma,[return] FROM transfers
        
        WHERE (strftime('%Y/%m/%d',[return]) > strftime('%Y/%m/%d',date('now','-7 days'))
            OR [return] IS NULL)
    '''
try:
    rl = pd.read_sql(q,conn)
#     display(rl)
    for rma in rl['RMA']:
        rma_list.append(rma)
except Exception as e:
    print(e)
len(rma_list)

# RMA LIST FROM TRANSFER
q = '''
        SELECT rma,[return] FROM completed
        
        WHERE (strftime('%Y/%m/%d',[return]) > strftime('%Y/%m/%d',date('now','-7 days'))
            OR [return] IS NULL)
    '''
try:
    rl = pd.read_sql(q,conn)
    display(rl)
    for rma in rl['RMA']:
        rma_list.append(rma)
except Exception as e:
    print(e)
len(rma_list)


,RMA,Return
0,FMSV2023060066,None
1,FMSV2023080005,None
2,FMSV2023080037,None
3,FMSV2023080040,None
4,FMSV2023080046,None
...,...,...
77,FMSV2022070004,None
78,FMSV2022060056,None
79,FMSV2022060072,None
80,FMSV2022060029,None


465

In [ ]:
'FMSV2023030009' in rma_list

In [5]:
# exfm_web
q = f'''
        SELECT DISTINCT
            c.[rma no.] AS rma,c.customer_name,c.customer_code AS customer,
            CASE
                WHEN e.location IS NULL THEN (SELECT e.location FROM consolidated c LEFT JOIN engineers e ON c.IN_INSPECT_USER_NAME = e.exfm_name)
                ELSE e.location END AS location,
            c.model AS model_d,c.serial_no AS sn,c.[Date Installed] AS install_date,
            c.[WARRANTY_END_DATE] AS warranty_date,
            c.[RECIEVE_DATE] AS receive_date,c.[IN_INSPECT_DATE] AS inspection_date,
            r.[start time] AS start_time,r.[end time] AS end_time,
            CASE
                WHEN c.[repair size] = 'Major' THEN 'Nặng'
                WHEN C.[repair size] = 'Minor' THEN 'Nhẹ'
                WHEN c.[repair size] IS NOT NULL THEN 'Khác' 
                ELSE c.[repair size] END AS repair_size,
            'Other' as issue_part,
            e.ks,c.[other id] AS wr_report,c.REPAIR_STATUS AS change_stt,c.approval,
            CASE
                WHEN c.approval  IN ('Decline','Cancel') THEN 'Chờ xác nhận (quá 3 tháng)'
                WHEN c.[create] < c.warranty_end_date AND c.repair_status ='Inspection' AND c.[other id] IS NULL THEN 'Chờ duyệt Bảo hành'
                WHEN c.[other id] IS NOT NULL AND c.repair_status ='Inspection' THEN 'Chuẩn bị linh kiện'
                ELSE s.vie_status END AS repair_status,
            CASE
            WHEN c.approval  IN ('Decline','Cancel') THEN 9
            ELSE s.stt_score END  AS e_score,
            CASE
                WHEN c.[other id] NOT NULL AND UPPER(c.[other id]) NOT LIKE '%REJECTED%' THEN 0.5
                ELSE 0 END AS w_score,
            
            c.[update time] AS update_time,c.Shipped as return_date

            FROM 
            ((consolidated c LEFT JOIN engineers e ON c.[update user] = e.exfm_code)
            LEFT JOIN status s ON c.[repair_status] = s.repair_status)
            LEFT JOIN repair_code r ON c.[rma no.] = r.[rma no.]
        WHERE c.[rma no.] IN {tuple(rma_list)}
    '''
try:
    exfm_web = pd.read_sql(q,conn)
#     display(exfm_web)
except Exception as e:
    print(e)


In [6]:
exfm_web.to_sql('exfm_web',conn,index=False,if_exists = 'replace')

465

In [ ]:
# pending_file new_ml
q = '''
        SELECT DISTINCT 
            m.rma AS rma_id,m.customer,m.model,m.sn,d.dealer,scopes.vie AS type_s,
            STRFTIME('%Y-%m-%d',m.inspection) AS tr_date,
            STRFTIME('%Y-%m-%d',m.quotation) AS quotation_date,
            STRFTIME('%Y-%m-%d',m.confirmation) AS confirm_date,
            m.note as repair_note,m.status,status.vie_status as p_status,status.stt_score as p_score,
            m.return_date
            FROM ((new_ml m
                LEFT JOIN scopes ON m.model = scopes.model)
                LEFT JOIN status ON upper(m.status) = upper(status.repair_status))
                LEFT JOIN dealers d ON m.customer = d.customer
    '''
try:
    display(pd.read_sql(q,conn))
except Exception as e:
    print(e)

In [ ]:
# pending_file transfer
q = '''
        SELECT DISTINCT
            rma,tf.customer,tf.model,sn,d.dealer,
            scopes.vie as type_s,
            STRFTIME('%Y-%m-%d',tf.tr_date) as tr_date,
            STRFTIME('%Y-%m-%d',tf.quotation_date) as quotation_date,
            STRFTIME('%Y-%m-%d',tf.confirmation) as confirm_date,
            tf.note as repair_note,
            CASE
            WHEN tf.clear IS NULL THEN 'transfer table'
            ELSE tf.clear END as status,
            CASE
                WHEN tf.return is null THEN
                (CASE
                    WHEN s.vie_status ="Chờ xác nhận" THEN "Chờ xác nhận (quá 3 tháng)"

                    ELSE s.vie_status END)
                ELSE 'Đã trả hàng (không sửa)' END AS p_status,
             CASE
                WHEN tf.return is null THEN	
                (CASE
                    WHEN s.stt_score = "3.0" THEN 9
                    ELSE s.stt_score END )
                ELSE 10 END AS p_score,
            (strftime('%Y-%m-%d',tf.return)) as return_date

            FROM ((transfers tf LEFT JOIN scopes ON tf.model = scopes.model)
                LEFT JOIN dealers d ON tf.customer = d.customer)
                LEFT JOIN status s ON upper(tf.old_status) = upper(s.repair_status)

            WHERE  RMA NOT NULL
            AND (strftime('%Y/%m/%d',tf.return) > strftime('%Y/%m/%d',date('now','-7 days'))
            OR tf.return IS NULL)
    '''
try:
    display(pd.read_sql(q,conn))
except Exception as e:
    print(e)

In [ ]:
# pending_file completed
q = '''
        SELECT DISTINCT
            rma,tf.customer,tf.model,sn,d.dealer,
            scopes.vie as type_s,
            STRFTIME('%Y-%m-%d',tf.quotation_date) as tr_date,
            STRFTIME('%Y-%m-%d',tf.quotation_date) as quotation_date,
            STRFTIME('%Y-%m-%d',tf.confirmation) as confirm_date,
            tf.note as repair_note,
            CASE
            WHEN tf.clear IS NULL THEN 'complete table'
            ELSE tf.clear END as status,
            CASE
                WHEN tf.return is null THEN 'Đang giao hàng'
                ELSE 'Hoàn tất giao hàng' END AS p_status,
            CASE
                WHEN tf.return is null THEN 7
                ELSE 8 END AS p_score,
            (strftime('%Y-%m-%d',tf.return)) as return_date

            FROM (completed tf LEFT JOIN scopes ON tf.model = scopes.model)
                LEFT JOIN dealers d ON tf.customer = d.customer
            WHERE  RMA NOT NULL
            AND (strftime('%Y/%m/%d',tf.return) > strftime('%Y/%m/%d',date('now','-7 days'))
            OR tf.return IS NULL)
    '''
try:
    display(pd.read_sql(q,conn))
except Exception as e:
    print(e)

In [ ]:
#new job 
q = '''
        SELECT DISTINCT rma as rma_id,ew.customer,model_d as model,sn,d.dealer,
            s.vie,strftime('%Y-%m-%d',inspection_date) as tr_date,
            CASE
            WHEN rma IS NULL THEN NULL
            ELSE NULL END AS quotation_date,
            CASE
            WHEN rma IS NULL THEN NULL
            ELSE NULL END AS confirm_date,
            CASE
            WHEN rma IS NULL THEN NULL
            ELSE NULL END AS note,
            change_stt as status,st.vie_status as p_status,st.stt_score as p_score,(strftime('%Y-%m-%d',ew.return_date)) as return_date


            FROM ((exfm_web ew
            LEFT JOIN dealers d ON d.customer = ew.customer)
            LEFT JOIN scopes s ON s.model = ew.model_d)
            LEFT JOIN status st ON st.repair_status = ew.change_stt

            WHERE rma > (SELECT max(rma) FROM new_ml)
    '''
try:
    display(pd.read_sql(q,conn))
except Exception as e:
    print(e)

In [7]:
# pending file total
q = '''
        SELECT DISTINCT 
            m.rma AS rma_id,m.customer,m.model,m.sn,d.dealer,scopes.vie AS type_s,
            STRFTIME('%Y-%m-%d',m.inspection) AS tr_date,
            STRFTIME('%Y-%m-%d',m.quotation) AS quotation_date,
            STRFTIME('%Y-%m-%d',m.confirmation) AS confirm_date,
            m.note as repair_note,m.status,status.vie_status as p_status,status.stt_score as p_score,
            STRFTIME('%Y-%m-%d',m.return_date) AS return_date
            FROM ((new_ml m
                LEFT JOIN scopes ON m.model = scopes.model)
                LEFT JOIN status ON upper(m.status) = upper(status.repair_status))
                LEFT JOIN dealers d ON m.customer = d.customer
        UNION ALL
            SELECT DISTINCT
            rma,tf.customer,tf.model,sn,d.dealer,
            scopes.vie as type_s,
            STRFTIME('%Y-%m-%d',tf.tr_date) as tr_date,
            STRFTIME('%Y-%m-%d',tf.quotation_date) as quotation_date,
            STRFTIME('%Y-%m-%d',tf.confirmation) as confirm_date,
            tf.note as repair_note,
            CASE
            WHEN tf.clear IS NULL THEN 'transfer table'
            ELSE tf.clear END as status,
            CASE
                WHEN tf.return is null THEN
                (CASE
                    WHEN s.vie_status ="Chờ xác nhận" THEN "Chờ xác nhận (quá 3 tháng)"

                    ELSE s.vie_status END)
                ELSE 'Đã trả hàng (không sửa)' END AS p_status,
             CASE
                WHEN tf.return is null THEN	
                (CASE
                    WHEN s.stt_score = "3.0" THEN 9
                    ELSE s.stt_score END )
                ELSE 10 END AS p_score,
            (strftime('%Y-%m-%d',tf.return)) as return_date

            FROM ((transfers tf LEFT JOIN scopes ON tf.model = scopes.model)
                LEFT JOIN dealers d ON tf.customer = d.customer)
                LEFT JOIN status s ON upper(tf.old_status) = upper(s.repair_status)

            WHERE  RMA NOT NULL
            AND (strftime('%Y/%m/%d',tf.return) > strftime('%Y/%m/%d',date('now','-7 days'))
            OR tf.return IS NULL)
        
        UNION ALL
             SELECT DISTINCT
            rma,tf.customer,tf.model,sn,d.dealer,
            scopes.vie as type_s,
            STRFTIME('%Y-%m-%d',tf.quotation_date) as tr_date,
            STRFTIME('%Y-%m-%d',tf.quotation_date) as quotation_date,
            STRFTIME('%Y-%m-%d',tf.confirmation) as confirm_date,
            tf.note as repair_note,
            CASE
            WHEN tf.clear IS NULL THEN 'complete table'
            ELSE tf.clear END as status,
            CASE
                WHEN tf.return is null THEN 'Đang giao hàng'
                ELSE 'Hoàn tất giao hàng' END AS p_status,
            CASE
                WHEN tf.return is null THEN 7
                ELSE 8 END AS p_score,
            (strftime('%Y-%m-%d',tf.return)) as return_date

            FROM (completed tf LEFT JOIN scopes ON tf.model = scopes.model)
                LEFT JOIN dealers d ON tf.customer = d.customer
            WHERE  RMA NOT NULL
            AND (strftime('%Y/%m/%d',tf.return) > strftime('%Y/%m/%d',date('now','-7 days'))
            OR tf.return IS NULL)
        UNION ALL
            SELECT DISTINCT rma as rma_id,ew.customer,model_d as model,sn,d.dealer,
            s.vie,strftime('%Y-%m-%d',inspection_date) as tr_date,
            CASE
            WHEN rma IS NULL THEN NULL
            ELSE NULL END AS quotation_date,
            CASE
            WHEN rma IS NULL THEN NULL
            ELSE NULL END AS confirm_date,
            CASE
            WHEN rma IS NULL THEN NULL
            ELSE NULL END AS note,
            change_stt as status,st.vie_status as p_status,st.stt_score as p_score,(strftime('%Y-%m-%d',ew.return_date)) as return_date


            FROM ((exfm_web ew
            LEFT JOIN dealers d ON d.customer = ew.customer)
            LEFT JOIN scopes s ON s.model = ew.model_d)
            LEFT JOIN status st ON st.repair_status = ew.change_stt

            WHERE rma > (SELECT max(rma) FROM new_ml)
    '''
try:
    pending_file = pd.read_sql(q,conn)
    display(pending_file)
    
except Exception as e:
    print(e)
    


,rma_id,customer,model,sn,dealer,type_s,tr_date,quotation_date,confirm_date,repair_note,status,p_status,p_score,return_date
0,FMSV2023020019,"FUJIFILM Vietnam Co., Ltd.",EC-760ZP-V/M,2C730K027,None,Ống soi đại tràng,2023-02-13,None,None,Part OK: 14-Aug. #PO109,waiting for repair,None,NaN,None
1,FMSV2023020022,FTYO - Loaner Asset,EC-760ZP-V/M,2C730K032,None,Ống soi đại tràng,2023-02-14,None,None,Part OK: 14-Aug. #PO109,waiting for repair,None,NaN,None
2,FMSV2023020034,Hoang Long Clinic,EC-760R-V/I,1C728K180,Trang Thi,Ống soi đại tràng,2023-02-23,2023-03-22,None,PO111,waiting for part,Chuẩn bị linh kiện,4.0,None
3,FMSV2023020050,Hanoi Medical University Hospital,EC-600WI,1C692K487,Trang Thi,Ống soi đại tràng,2023-02-21,2023-02-27,2023-02-27,Confirmed: 27-Feb.,Completed repair,Hoàn tất sửa chữa,7.0,None
4,FMSV2023020067,Hanoi Medical University Hospital,VP-4450HD,2V567K684,Trang Thi,Bộ Xử lý,2023-03-02,2023-03-15,2023-03-29,None,LP/waiting for part,Chuẩn bị linh kiện,4.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
460,FMSV2022070004,Hanoi Medical University Hospital,EC-600WI,5C692K028,Trang Thi,Ống soi đại tràng,None,None,None,None,complete table,Đang giao hàng,7.0,None
461,FMSV2022060056,Hanoi Medical University Hospital,EG-600WR,2G391K085,Trang Thi,Ống soi dạ dày,None,None,None,None,complete table,Đang giao hàng,7.0,None
462,FMSV2022060072,Hanoi Medical University Hospital,EC-600WI,1C692K733,Trang Thi,Ống soi đại tràng,None,None,None,None,complete table,Đang giao hàng,7.0,None
463,FMSV2022060029,Hanoi Medical University Hospital,EC-600WI,3C692K098,Trang Thi,Ống soi đại tràng,2022-06-16,2022-06-16,None,1. Less than 4 weeks after giving the quotation,complete table,Đang giao hàng,7.0,None


In [8]:
# part_name
q = f'''
        SELECT 
            DISTINCT p.[RMA No.] AS rma,p.PART_NO AS part_no,
            p.Part_description AS part_description,pn.vie as vie_name
            FROM parts p
            LEFT JOIN part_name pn ON p.part_description = pn.part_description

            WHERE rma IN {tuple(rma_list)}
        ORDER BY rma
    '''
try:
    parts = pd.read_sql(q,conn)
    display(parts)
except Exception as e:
    print(e)

,rma,part_no,part_description,vie_name
0,FMSV2019080079,343A1295408G,LG FLEXIBLE SECTION ASSY ROHS,Vỏ ống dẫn
1,FMSV2019080079,342A1318071B,BSA-C287A,Bộ phận uốn cong
2,FMSV2019080079,68B12078100,RUBBER FOR BSA ROHS,Vỏ cao su
3,FMSV2019080079,54A3792370,NUT ASSY ROHS,Đai ốc (đồng)
4,FMSV2019080079,42B593410A,SLEEVE ROHS,Ống nối (đồng)
...,...,...,...,...
3080,FMSV2023080088,370Y200590_,AW TUBE ASS''Y,Kênh khí nước
3081,FMSV2023080088,68B12952250,WATER JET TUBE,Kênh nước phụ
3082,FMSV2023080088,54A379237A,NUT ASS'Y,Đai ốc (đồng)
3083,FMSV2023080088,42B593410A,PIPE SLEEVE,Ống nối (đồng)


In [9]:
# r_code
q = f'''
        SELECT r.[rma no.] as rma,pf.r_code as r_code,
        SUBSTRING(pf.r_description,1,50) as r_description,
        pf.vie_name

        FROM r_code pf
        LEFT JOIN repair_code r ON pf.r_code = r.r_code
        WHERE rma IN {tuple(rma_list)}
    '''
try:
    r_code = pd.read_sql(q,conn)
    display(r_code)
except Exception as e:
    print(e)

,rma,r_code,r_description,vie_name
0,FMSV2022120042,R002,ANGL. Replace LR Lock Knob,Bộ phận điều khiển
1,FMSV2023020047,R002,ANGL. Replace LR Lock Knob,Bộ phận điều khiển
2,FMSV2023070009,R002,ANGL. Replace LR Lock Knob,Bộ phận điều khiển
3,FMSV2023070044,R002,ANGL. Replace LR Lock Knob,Bộ phận điều khiển
4,FMSV2023070079,R002,ANGL. Replace LR Lock Knob,Bộ phận điều khiển
...,...,...,...,...
1304,FMSV2023060059,R006,AWB.Replace O-Ring/One Way Valve,Vòng cao su
1305,FMSV2023070054,R006,AWB.Replace O-Ring/One Way Valve,Vòng cao su
1306,FMSV2023050042,R322,PCB. Replace Main CPU PCB Assembly,Thay thế bo mạch chính
1307,FMSV2023050081,R263,FUS.Replace Fuse,Thay cầu chì


In [ ]:
today = dt.now().strftime('%y%m%d')
pending_name = f'pending_{today}.xls'
pending_file.to_excel(f'New Web/{pending_name}',index = False)
exfm_name = f'exfm_web_{today}.xls'
exfm_web.to_excel(f'New Web/{exfm_name}',index = False)
parts_name = f'part_name_{today}.xls'
parts.to_excel(f'New Web/{parts_name}',index = False)
r_code_name = f'r_code_{today}.xls'
r_code.to_excel(f'New Web/{r_code_name}',index = False)


In [13]:
today = dt.now().strftime('%y%m%d')
folder_name = 'New web'
pending_name = f'pending_{today}.xls'
exfm_name = f'exfm_web_{today}.xls'
parts_name = f'part_name_{today}.xls'
r_code_name = f'r_code_{today}.xls'

# self.pending_file.to_excel(f'New Web/{pending_name}',index = False)
# self.exfm_web.to_excel(f'New Web/{exfm_name}',index = False)
# self.parts.to_excel(f'New Web/{parts_name}',index = False)
# self.r_code.to_excel(f'New Web/{r_code_name}',index = False)
writer = pd.ExcelWriter(f'{folder_name}/{pending_name}')
pending_file.to_excel(writer,sheet_name='Sheet1',index=False)
writer = pd.ExcelWriter(f'{folder_name}/{exfm_name}')
exfm_web.to_excel(writer,sheet_name='Sheet1',index=False)
writer = pd.ExcelWriter(f'{folder_name}/{parts_name}')
parts.to_excel(writer,sheet_name='Sheet1',index=False)
writer = pd.ExcelWriter(f'{folder_name}/{r_code_name}')
r_code.to_excel(writer,sheet_name='Sheet1',index=False)


C:\Users\quang\AppData\Local\Temp\ipykernel_15236\746407130.py:12: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  writer = pd.ExcelWriter(f'{folder_name}/{pending_name}')
C:\Users\quang\AppData\Local\Temp\ipykernel_15236\746407130.py:14: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and w

In [ ]:
# exfm_web
q = '''
        SELECT * FROM r_code
    '''
try:
    display(pd.read_sql(q,conn))
except Exception as e:
    print(e)

In [ ]:
from sqlite3 import connect
conn = connect('history.db')

In [ ]:
import sources.new_web as nw

In [ ]:
run_all = nw.data_process(conn)
run_all.rma_list()
run_all.exfm_web()
run_all.pending_file()
run_all.parts_name()
run_all.r_code()
run_all.export_files()

In [ ]:
!pip freeze
